In [ ]:
from fastai.text.all import *

In [ ]:
lang = 'de'
wiki = f'{lang}wiki'
base_path = Path('data')
path = base_path/wiki
data_path = path/'germeval'
class_path = path/'model'/'class'

# Load classification learners

In [ ]:
path_fwd = f'{class_path}/fwd/export.pkl'
learn_fwd = load_learner(path_fwd)

In [ ]:
path_bwd = f'{class_path}/bwd/export.pkl'
learn_bwd = load_learner(path_bwd)

# Get predictions for simple texts

```
Text: Komisch das die Realitätsverweigerung immer von linken erbärmlichen Correctiv Accounts ausgeht...  
label: OFFENSE  
label_fine: INSULT
```

In [ ]:
text = 'Komisch das die Realitätsverweigerung immer von linken erbärmlichen Correctiv Accounts ausgeht...'
pred = learn_fwd.predict(text)
pred

('OFFENSE', tensor(0), tensor([0.9024, 0.0976]))

# Evaluation on GermEval2019 Task 2 (Fine)

## Load GermEval2019 data

In [ ]:
names = ['text','label','label_fine']

In [ ]:
df_test = pd.read_csv(data_path/'germeval2019/germeval2019GoldLabelsSubtask1_2.txt',
                sep ='\t', names=names, quoting=3)

In [ ]:
def clean_text(text):
    text = re.sub('@\w+', '', text)
    text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', "", text)
    text = text.replace('|LBR|', ' ')
    text = text.replace('\"', ' ')
    text = ' '.join(text.split())
    return text
df_test['text'] = df_test['text'].apply(clean_text)

## Forward model

In [ ]:
dl_fwd = learn_fwd.dls.test_dl(df_test, with_labels=True)
preds_fwd = learn_fwd.get_preds(dl=dl_fwd)
accuracy(*preds_fwd)

TensorBase(0.7958)

## Backward model

In [ ]:
dl_bwd = learn_bwd.dls.test_dl(df_test, with_labels=True)
preds_bwd = learn_bwd.get_preds(dl=dl_bwd)
accuracy(*preds_bwd)

TensorBase(0.8001)

## Ensemble Forward + Backward

In [ ]:
avg = 'macro'
precision = Precision(average=avg)
recall = Recall(average=avg)
f1score = F1Score(average=avg)

In [ ]:
preds = (preds_fwd[0] + preds_bwd[0]) / 2
a = accuracy(preds, preds_fwd[1])
a

TensorBase(0.8034)

In [ ]:
p = precision(torch.argmax(preds, axis=1), preds_fwd[1])
p

0.7781402318624189

In [ ]:
r = recall(torch.argmax(preds, axis=1), preds_fwd[1])
r

0.7571792293801928

In [ ]:
f1 = f1score(torch.argmax(preds, axis=1), preds_fwd[1])
f1

0.7657193813923859

Save stats

In [ ]:
stats = {
    'accuracy': float(a),
    'precision': p,
    'recall': r,
    'f1score': f1
}

with open(f'{class_path}/inference_stats.json', 'w') as f:
    json.dump(stats, f, ensure_ascii=False, indent=4)

# Interpreation with fastinference

see [fastinference](https://muellerzr.github.io/fastinference/)

`intrinsic_attention()` shows which tokens contribute most to the classification.   
Red tokens = small contribution  
Grenn tokens = high contribution

In [ ]:
from fastinference.inference.text import intrinsic_attention
learn_fwd.intrinsic_attention(text)

In [ ]:
learn_fwd.predict(text)

[['OFFENSE'], array([[0.90238917, 0.09761085]], dtype=float32)]